In [1]:
"""
# this is the first version ranker
# first, load the embeddings
# stored in torch tensors

import torch
import numpy as np
import os
import sys
import json
import argparse
import time
import random
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import scipy.stats 

embeddings_folder = '../data/cifar-10-embedding-3/embeddings/' 
embeddings_files = sorted(os.listdir(embeddings_folder))
test_embeddings = torch.load(embeddings_folder + embeddings_files[0]) # load the first file
print(test_embeddings.size()) # 128, 32, 32, 32

# calculate the entropy of the embeddings, select 1%, 5%, 10%, 20%, 50% of the embeddings
def calculate_entropy(embs, percentage):
    # embs are torch tensors
    # percentage shows the number of embeddings to select
    embs_entropy = []
    for i in range(embs.size(len(embs.size())-3)):
        embs_entropy.append(scipy.stats.entropy(embs[:, i].reshape(-1))) # b, c
    # get the top percentage of the channels
    embs_entropy = np.array(embs_entropy)
    indices = np.argsort(embs_entropy)
    num_selected = int(embs.size(1) * percentage)
    selected_indices = indices[:num_selected]
    return selected_indices
    # out b, c*p, h, w or c*p, h, w

def ranker_entropy(embs, percentage):
    # calculate the entropy of the embeddings
    selected_indices = calculate_entropy(embs, percentage)
    # select indice from test embeddings
    selected_embeddings = embs[:, selected_indices] # b, c*p, h, w
    return selected_embeddings

# get the selected embeddings
selected_embeddings = ranker_entropy(test_embeddings, 0.1)
print(selected_embeddings.size())

# design a simple gating mechanism for regression
"""

"\n# this is the first version ranker\n# first, load the embeddings\n# stored in torch tensors\n\nimport torch\nimport numpy as np\nimport os\nimport sys\nimport json\nimport argparse\nimport time\nimport random\nimport torch.nn as nn\nimport torch.optim as optim\nfrom torch.autograd import Variable\nimport scipy.stats \n\nembeddings_folder = '../data/cifar-10-embedding-3/embeddings/' \nembeddings_files = sorted(os.listdir(embeddings_folder))\ntest_embeddings = torch.load(embeddings_folder + embeddings_files[0]) # load the first file\nprint(test_embeddings.size()) # 128, 32, 32, 32\n\n# calculate the entropy of the embeddings, select 1%, 5%, 10%, 20%, 50% of the embeddings\ndef calculate_entropy(embs, percentage):\n    # embs are torch tensors\n    # percentage shows the number of embeddings to select\n    embs_entropy = []\n    for i in range(embs.size(len(embs.size())-3)):\n        embs_entropy.append(scipy.stats.entropy(embs[:, i].reshape(-1))) # b, c\n    # get the top percentage o

In [1]:
import torch
import numpy as np
import os
import sys
import json
import argparse
import time
import random
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import scipy.stats 

def calculate_entropy(embs, percentage):
    # embs are torch tensors
    # percentage shows the number of embeddings to select
    embs_entropy = []
    for i in range(embs.size(len(embs.size())-3)):
        embs_entropy.append(scipy.stats.entropy(embs[:, i].reshape(-1))) # b, c
    # get the top percentage of the channels
    embs_entropy = np.array(embs_entropy)
    indices = np.argsort(embs_entropy)
    num_selected = int(embs.size(1) * percentage)
    selected_indices = indices[:num_selected]
    return selected_indices
    # out b, c*p, h, w or c*p, h, w

def ranker_entropy(embs, percentage):
    # calculate the entropy of the embeddings
    selected_indices = calculate_entropy(embs, percentage)
    # select indice from test embeddings
    selected_embeddings = embs[:, selected_indices] # b, c*p, h, w
    return selected_embeddings, selected_indices


In [4]:
import sys
import json
import argparse
import time
import random
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import scipy.stats
import torch.utils 
import os
from Models import mobilenetv2


In [3]:
# import the server model
run_device = 'tintin'
# This is the version 1 gated regression model, we need a simpler version
# class GatedRegression(nn.Module):
#     def __init__(self, input_size, num_classes):
#         super(GatedRegression, self).__init__()
#         # think about it 3*32*32 -> 1
#         # think about the classification of the mobile net
#         self.input_size = input_size
#         self.num_classes = num_classes
#         self.hidden_size = input_size
#         self.conv1 = nn.Conv2d(input_size, 2*self.hidden_size, kernel_size=3, stride=2, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(2*self.hidden_size)
#         self.conv2 = nn.Conv2d(2*self.hidden_size, 4*self.hidden_size, kernel_size=3, stride=2, padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(4*self.hidden_size)
#         self.conv3 = nn.Conv2d(4*self.hidden_size, 8*self.hidden_size, kernel_size=3, stride=2, padding=1, bias=False)
#         self.bn3 = nn.BatchNorm2d(8*self.hidden_size)
#         self.linear = nn.Linear(8*self.hidden_size, num_classes)
#         self.relu = nn.ReLU(inplace=True)
#         self.avgpool = nn.AvgPool2d(4)
#         self.flatten = nn.Flatten(start_dim=1, end_dim=-1)

#     def forward(self, x):
#         # need to change 32 -> 4
#         out = self.conv1(x)
#         out = self.bn1(out)
#         out = self.conv2(out)
#         out = self.bn2(out)
#         out = self.conv3(out)
#         out = self.bn3(out)
#         out = self.relu(out)
#         out = self.avgpool(out)
#         out = self.flatten(out)
#         out = self.linear(out)
#         return out

class GatedRegression(nn.Module):
    def __init__(self, input_size, weight, height):
        super(GatedRegression, self).__init__()
        # think about it 3*32*32 -> 1
        # think about the classification of the mobile net
        # the input size is b, c*p, h, w, the output size is b 
        # how to make sure more features help the server model?

        self.input_size = input_size * weight * height
        self.structure = [self.input_size, self.input_size//weight, self.input_size//height//weight, 1]
        self.linear1 = nn.Linear(self.structure[0], self.structure[1])
        self.linear2 = nn.Linear(self.structure[1], self.structure[2])
        self.linear3 = nn.Linear(self.structure[2], self.structure[3])
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(0.3)
        self.flatten = nn.Flatten(start_dim=1, end_dim=-1)

    def forward(self, x):
        # need to change it to 0-1
        # flatten the input first
        out = self.flatten(x)
        out = self.linear1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.linear2(out)
        return out

# load the dataset
class gated_dataset(torch.utils.data.Dataset):
    # load the dataset
    def __init__(self, embeddings_folder):
        self.embeddings_folder = embeddings_folder
        self.embeddings_files = sorted(os.listdir(embeddings_folder+'/embeddings/'))
        self.labels_files = sorted(os.listdir(embeddings_folder+'/labels/'))

    def __len__(self):
        return self.embeddings_files.__len__()

    def __getitem__(self, idx):
        if idx == 0:
            self.embeddings = torch.load(self.embeddings_folder+'/embeddings/' + self.embeddings_files[idx])
            self.labels = torch.load(self.embeddings_folder+'/labels/' + self.labels_files[idx])
        else:
            self.embeddings = torch.cat((self.embeddings, torch.load(embeddings_folder+'/embeddings/' + self.embeddings_files[idx])), 0)
            self.labels = torch.cat((self.labels, torch.load(embeddings_folder+'/labels/' + self.labels_files[idx])), 0)
        return self.embeddings, self.labels
    
# load the dataset
if run_device == 'tintin':
    embeddings_folder = '/data/anp407/cifar-10-embedding-3/'
if run_device == 'server':
    embeddings_folder = '../data/cifar-10-embedding-3/'
dataset = gated_dataset(embeddings_folder)
# print(gated_dataset.__len__(dataset)) # 391 batches                                             

# load the data loader, train no test
train_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

# load the model
Gated = GatedRegression(input_size=3, weight=32, height=32)
Gated = Gated.cuda()
Gated.train()

# get the server model
client_model, server_model = mobilenetv2.stupid_model_splitter(weight_path='./Weights/cifar-10/MobileNetV2.pth')
server_model = server_model.cuda()
server_model.eval()
# load the optimizer
optimizer = optim.Adam(Gated.parameters(), lr=0.001)

# load the loss function
criterion = nn.CrossEntropyLoss()

# train the model
for epoch in range(100):
    for i, data in enumerate(train_loader):
        embs, labels = data
        # for the training, embeddings doesn't need to be on the cuda
        # embs, labels = embs.cuda(), labels.cuda()
        labels = labels.cuda()
        # embeddings = Variable(embeddings) # what are they?
        # labels = Variable(labels) # what are they?
        embs = embs.squeeze(0)
        labels = labels.squeeze(0)
        s_embs, s_inds = ranker_entropy(embs, 0.1)
        s_embs = s_embs.cuda()
        # make the embeddings to fit the server model
        n_embs = torch.zeros(s_embs.size(0), 32, 32, 32)
        n_embs[:, s_inds] = s_embs
        n_embs = n_embs.cuda()

        # get the output from the server model
        with torch.no_grad():
            outputs = server_model(n_embs)

        outputs = Gated(selected_embeddings)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' % (epoch+1, 10, i+1, len(train_loader), loss.item()))

torch.Size([128, 32, 32, 32])


../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [5,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [6,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_f

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
